In [1]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../"))

import desc
from desc.objectives import *
from desc.basis import ZernikePolynomial, FourierZernikeBasis, DoubleFourierSeries
from desc.transform import Transform
from desc.equilibrium import EquilibriaFamily, Equilibrium
from desc.plotting import plot_1d,plot_2d,plot_3d,plot_comparison, plot_section, plot_fsa, plot_surfaces
from desc.optimize import Optimizer
from desc.perturbations import *
import numpy as np
from desc.geometry import FourierRZToroidalSurface, ZernikeRZToroidalSection
%matplotlib inline
from desc.utils import copy_coeffs
import matplotlib.pyplot as plt
from desc.examples import get
from desc.objectives.getters import get_fixed_boundary_constraints, maybe_add_self_consistency
from desc.grid import LinearGrid, QuadratureGrid, Grid
import plotly.graph_objects as go
from desc.profiles import PowerSeriesProfile
from desc.__main__ import main
from desc.vmec_utils import vmec_boundary_subspace

DESC version 0.11.1+573.ga8c221939.dirty,using JAX backend, jax version=0.4.25, jaxlib version=0.4.25, dtype=float64
Using device: CPU, with 10.17 GB available memory


In [14]:
from test_utils import area_difference, compute_coords

# solve an equilibrium with R,Z and lambda specified on zeta=0 surface
eq = get("SOLOVEV")
eq_poin = eq.set_poincare_equilibrium()

eq_poin.change_resolution(
    eq_poin.L, eq_poin.M, 1, N_grid=2
)  # add toroidal modes to the equilibrium

eq.change_resolution(
    eq_poin.L, eq_poin.M, 1, N_grid=2
) 

# perturb slightly from the axisymmetric equilibrium
eq_poin.R_lmn = eq_poin.R_lmn.at[eq_poin.R_basis.get_idx(1, 1, 1)].set(0.1)
# this constrains lambda at the zeta=0 surface, using eq's current value of lambda
constraints = get_fixed_xsection_constraints(eq=eq_poin)
objective = ObjectiveFunction(ForceBalance(eq=eq_poin))
eq_poin.solve(
    verbose=3,
    ftol=1e-10,
    objective=objective,
    constraints=constraints,
    maxiter=100,
    xtol=1e-10,
    gtol=1e-10,
)

Building objective: force
Precomputing transforms
Timer: Precomputing transforms = 93.7 ms
Timer: Objective build = 282 ms
Building objective: poincare R
Building objective: poincare Z
Building objective: poincare Lambda
Building objective: fixed-Psi
Building objective: fixed-pressure
Building objective: fixed-iota
Building objective: self_consistency section R
Building objective: self_consistency section Z
Building objective: self_consistency section Lambda
Building objective: self_consistency boundary R
Building objective: self_consistency boundary Z
Building objective: lambda gauge
Building objective: self_consistency axis R
Building objective: self_consistency axis Z
Timer: Objective build = 908 ms
Timer: Linear constraint projection build = 2.55 sec
Number of parameters: 507
Number of objectives: 1900
Starting optimization
Using method: lsq-exact
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1          1.051e+01   

(Equilibrium at 0x776d6c8e3fb0 (L=24, M=12, N=1, NFP=1, sym=1, spectral_indexing=fringe),
     message: `gtol` condition satisfied.
     success: True
         fun: [ 1.037e-10 -5.725e-10 ... -1.465e-07 -7.333e-08]
           x: [-2.399e-06  5.110e-07 ...  4.767e-03  7.900e-01]
         nit: 62
        cost: 1.2112132906951991e-12
           v: [ 1.000e+00  1.000e+00 ...  1.000e+00  1.000e+00]
  optimality: 7.572150003830413e-11
        nfev: 71
        njev: 63
        allx: [Array([-1.010e-15, -3.449e-16, ...,  0.000e+00,  0.000e+00],      dtype=float64), Array([ 1.699e-04, -1.514e-03, ...,  0.000e+00,  0.000e+00],      dtype=float64), Array([ 1.099e-03,  6.847e-03, ...,  0.000e+00,  0.000e+00],      dtype=float64), Array([ 1.292e-03,  1.657e-02, ...,  0.000e+00,  0.000e+00],      dtype=float64), Array([ 8.979e-04,  2.172e-02, ...,  0.000e+00,  0.000e+00],      dtype=float64), Array([ 5.933e-04,  2.395e-02, ...,  0.000e+00,  0.000e+00],      dtype=float64), Array([-1.675e-04,  2.862e

In [15]:
Rr1, Zr1, Rv1, Zv1 = compute_coords(eq, Nz=6)
Rr2, Zr2, Rv2, Zv2 = compute_coords(eq_poin, Nz=6)
rho_err, theta_err = area_difference(Rr1, Rr2, Zr1, Zr2, Rv1, Rv2, Zv1, Zv2)

print(f"rho_err = {rho_err}")
print(f"theta_err = {theta_err}")
np.testing.assert_allclose(rho_err, 0, atol=6e-3)
np.testing.assert_allclose(theta_err, 0, atol=2e-3)

grid = LinearGrid(L=50, M=50, zeta=0)
L_2D = eq.compute(names="lambda", grid=grid)
L_3D = eq_poin.compute(names="lambda", grid=grid)
np.testing.assert_allclose(L_2D["lambda"], L_3D["lambda"], atol=1e-15)

rho_err = [[4.94012310e-14 0.00000000e+00 8.63312077e-15 1.26189775e-14
  8.81782148e-15 7.14069395e-15 7.73203002e-15 7.68547353e-15
  6.84193172e-15 1.77017589e-14]
 [1.76104292e-03 9.54228487e-04 7.32317599e-04 6.70848066e-04
  6.43335874e-04 6.28762100e-04 6.20917222e-04 6.17360209e-04
  6.17376336e-04 6.22898142e-04]
 [4.18537269e-03 2.07242014e-03 1.36279215e-03 1.03681959e-03
  8.95028848e-04 8.18844729e-04 7.74596963e-04 7.48884598e-04
  7.36690475e-04 7.45864662e-04]
 [5.42634902e-03 2.69967353e-03 1.78494341e-03 1.32141866e-03
  1.04060607e-03 8.58932068e-04 7.42982736e-04 6.70806164e-04
  6.28109771e-04 6.46714124e-04]
 [4.18537269e-03 2.07242014e-03 1.36279215e-03 1.03681959e-03
  8.95028848e-04 8.18844729e-04 7.74596963e-04 7.48884598e-04
  7.36690475e-04 7.45864662e-04]
 [1.76104292e-03 9.54228487e-04 7.32317599e-04 6.70848066e-04
  6.43335874e-04 6.28762100e-04 6.20917222e-04 6.17360209e-04
  6.17376336e-04 6.22898142e-04]]
theta_err = [[0.00000000e+00 4.85387596e-17 6.1